# < Tensorflow 2.0 >

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

## 1. Preprocess

In [12]:
(train_x,train_y),(test_x,test_y) = datasets.mnist.load_data()

In [13]:
train_x  = train_x[...,tf.newaxis] # channel 만들어주기
test_x = test_x[...,tf.newaxis]

train_x = train_x/255. # scaling
test_x = test_x/255.

## 2. Modeling

In [5]:
input_shape = (28,28,1)
num_classes = 10

lr = 0.001

In [15]:
inputs = layers.Input(input_shape) # input의 shape을 지정해줘야 한다

# Feature Extraction
net = layers.Conv2D(32,(3,3), padding='SAME')(inputs) # 32개의 3*3필터
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Conv2D(64,(3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(0.5)(net)

# FC
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
              loss='sparse_categorical_crossentropy',
             metrics=[tf.keras.metrics.Accuracy()]) 

## 3. Training

In [17]:
num_epochs = 1
batch_size = 64

# < Pytorch >

In [19]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [20]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size=64
test_batch_size=64

epochs = 3

no_cuda = False
log_interval = 100  #pytorch에서는 progress bar가 나타나지 않으므로 직접해줘야!

## 1. Modeling

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,20,5,1) # in : 1개 / out : 20개 / 5*5 크기의 1개 channel
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1 = nn.Linear(4*4*50,500)
        self.fc2 = nn.Linear(500,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

## 2. Preprocess

In [31]:
datasets.MNIST('../dataset', train=True, download=True,
              transform=transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,),(0.5,))
              ]))

torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda. is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' :1, 'pin_memory' : True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
              transform=transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,),(0.5,))
              ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
              transform=transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,),(0.5,))
              ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

## 3. Optimization

In [32]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

## 4. Training

In [33]:
for epoch in range(1,epochs+1):
    
    # Train mode
    model.train()
    
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backprop 계산 전, 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward() # backprop한다
        optimizer.step() # update한다
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100*batch_idx / len(train_loader), loss.item()))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 속도 UP
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch_loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item() # sum up correct numbers
            
    test_loss /= len(test_loader.dataset)
    
    print("\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%)\n".format(
        test_loss, correct, len(test_loader.dataset),
        100*correct / len(test_loader.dataset)))

Train Epoch : 1 [0/60000 (0%)]	Loss : 2.301638
Train Epoch : 1 [6400/60000 (11%)]	Loss : 0.104958
Train Epoch : 1 [12800/60000 (21%)]	Loss : 0.100281
Train Epoch : 1 [19200/60000 (32%)]	Loss : 0.061159
Train Epoch : 1 [25600/60000 (43%)]	Loss : 0.056963
Train Epoch : 1 [32000/60000 (53%)]	Loss : 0.088150
Train Epoch : 1 [38400/60000 (64%)]	Loss : 0.075661
Train Epoch : 1 [44800/60000 (75%)]	Loss : 0.089009
Train Epoch : 1 [51200/60000 (85%)]	Loss : 0.027800
Train Epoch : 1 [57600/60000 (96%)]	Loss : 0.113186

Test set : Average loss : 0.0376, Accuracy : 9874/10000 (99%)

Train Epoch : 2 [0/60000 (0%)]	Loss : 0.018750
Train Epoch : 2 [6400/60000 (11%)]	Loss : 0.088972
Train Epoch : 2 [12800/60000 (21%)]	Loss : 0.015538
Train Epoch : 2 [19200/60000 (32%)]	Loss : 0.015300
Train Epoch : 2 [25600/60000 (43%)]	Loss : 0.011674
Train Epoch : 2 [32000/60000 (53%)]	Loss : 0.005021
Train Epoch : 2 [38400/60000 (64%)]	Loss : 0.012510
Train Epoch : 2 [44800/60000 (75%)]	Loss : 0.012495
Train Epoch 